In [23]:
#%load_ext cudf.pandas
#!pip install cudf-cu11
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [48]:
from IPython.display import display
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.headless = True

browser = webdriver.Chrome(options=options)

team_mapping = {'Miami Dolphins':'mia', 'New York Jets':'nyj', 'Buffalo Bills':'buf', 'New England Patriots':'nwe',
                'Jacksonville Jaguars':'jax', 'Indianapolis Colts':'clt', 'Houston Texans':'htx', 'Tennessee Titans':'oti',
                'Cleveland Browns':'cle', 'Baltimore Ravens':'rav', 'Cincinnati Bengals':'cin', 'Pittsburgh Steelers': 'pit',
                'Las Vegas Raiders':'rai', 'Oakland Raiders': 'rai', 'Denver Broncos':'den', 'Kansas City Chiefs':'kan', 'Los Angeles Chargers':'sdg',
                'Dallas Cowboys':'dal', 'Philadelphia Eagles':'phi', 'Washington Commanders':'was',
                'New York Giants':'nyg','Atlanta Falcons':'atl', 'New Orleans Saints':'nor', 'Tampa Bay Buccaneers':'tam',
                'Carolina Panthers':'car','Detroit Lions':'det', 'Green Bay Packers':'gnb', 'Chicago Bears':'chi', 'Minnesota Vikings':'min',
                'Los Angeles Rams':'ram', 'San Francisco 49ers':'sfo', 'Arizona Cardinals':'crd', 'Seattle Seahawks':'sea'}
inverse_mapping = {v: k for k, v in team_mapping.items()}

def find_element_with_timeout(browser, by, value, timeout):
    wait = WebDriverWait(browser, timeout)
    element = wait.until(EC.presence_of_element_located((by, value)))
    return element

def conversion_rate(team, year):
    full_name = inverse_mapping[team]
    nickname = full_name.split()[-1]
    print(f'Scraping data for: {nickname}')
    browser = webdriver.Chrome()
    url = f'https://www.pro-football-reference.com/teams/{team}/{year}.htm' #{year}
    print(url)
    browser.get(url)
    table_element = find_element_with_timeout(browser, By.XPATH, f"//caption[text()='Schedule & Game Results Table']/parent::table", 1)
    table_html = table_element.get_attribute('outerHTML')
    schedule = pd.read_html(table_html)[0]
    schedule.to_csv('schedule.csv')
    date = schedule['Unnamed: 2_level_0']
    location = schedule['Unnamed: 8_level_0']
    opponent = schedule['Unnamed: 9_level_0']
    outcome = schedule['Unnamed: 5_level_0']
    points = schedule['Score']['Tm']
    points_allowed = schedule['Score'][['Opp']]
    pass_yards = schedule['Offense']['PassY']
    rush_yards = schedule['Offense']['RushY']
    turnovers = schedule['Offense']['TO']
    points_allowed.rename(columns = {'Opp': 'points_allowed'}, inplace = True)
    time_place = pd.concat([date, location, opponent, outcome, points, points_allowed, pass_yards, rush_yards, turnovers], axis = 1)
    time_place.rename(columns = {'Unnamed: 8_level_1':'location', 'Unnamed: 5_level_1': 'outcome'}, inplace = True)
    time_place['Date'] = pd.to_datetime(time_place['Date'] + f', {year}', errors='coerce')
    time_place.loc[(time_place['Date'] > pd.Timestamp(f'{year}-12-31')) | (time_place['Date'].dt.month.isin([1, 2])), 'Date'] += pd.DateOffset(years=1)
    time_place = time_place.rename(columns = {'Date':'date', 'Opp': 'opp', 'Tm': 'points', 'PassY': 'passing_yards', 'RushY': 'rushing_yards', 'TO': 'turnovers'})
    time_place['location'] = time_place['location'].replace('@', 'away').fillna('home')
    time_place = time_place.dropna(subset=['date'])
    time_place['date'] = time_place['date'].dt.strftime('%Y%m%d') + '0'
    time_place['opp'] = time_place['opp'].replace(team_mapping)
    time_place.to_csv('timeplace.csv')
    
    offensive_drives = pd.DataFrame()
    defensive_drives = pd.DataFrame()
    
    for d, l, o, r, p, pa, py, ry, t in zip(time_place.date, time_place.location, time_place.opp, time_place.outcome, time_place.points, time_place.points_allowed, time_place.passing_yards, time_place.rushing_yards, time_place.turnovers):
        try:
            if l == 'home':
                opponent_full_name = inverse_mapping[o]
                opponent_nickname = opponent_full_name.split()[-1]
                print(opponent_nickname)
                url = f'https://www.pro-football-reference.com/boxscores/{d}{team}.htm#home_drives'
                print(url)
                browser.get(url)
                otable_element = find_element_with_timeout(browser, By.XPATH, f"//caption[text()='{nickname} Drives Table']/parent::table", 10)
                otable_html = otable_element.get_attribute('outerHTML')
                dtable_element = find_element_with_timeout(browser, By.XPATH, f"//caption[text()='{opponent_nickname} Drives Table']/parent::table", 10)
                dtable_html = dtable_element.get_attribute('outerHTML')
                odrives = pd.read_html(otable_html)[0]
                ddrives = pd.read_html(dtable_html)[0]
                odrives['opponent'] = opponent_nickname
                ddrives['opponent'] = opponent_nickname
                odrives['date'] = d
                ddrives['date'] = d
                odrives['points'] = p #
                ddrives['points_allowed'] = pa #
                odrives['date'] = odrives['date'].astype(str).str.rstrip('0')
                ddrives['date'] = ddrives['date'].astype(str).str.rstrip('0')
                odrives['date'] = pd.to_datetime(odrives['date'], format='%Y%m%d')
                ddrives['date'] = pd.to_datetime(ddrives['date'], format='%Y%m%d')
                odrives['outcome'] = r
                ddrives['outcome'] = r
                odrives['passing_yards'] = py #
                odrives['rushing_yards'] = ry #
                odrives['turnovers'] = t #
                offensive_drives = pd.concat([offensive_drives, odrives])
                defensive_drives = pd.concat([defensive_drives, ddrives])
            elif l == 'away':
                opponent_full_name = inverse_mapping[o]
                opponent_nickname = opponent_full_name.split()[-1]
                print(opponent_nickname)
                url = f'https://www.pro-football-reference.com/boxscores/{d}{o}.htm#home_drives'
                print(url)
                browser.get(url)
                otable_element = find_element_with_timeout(browser, By.XPATH, f"//caption[text()='{nickname} Drives Table']/parent::table", 10)
                otable_html = otable_element.get_attribute('outerHTML')
                dtable_element = find_element_with_timeout(browser, By.XPATH, f"//caption[text()='{opponent_nickname} Drives Table']/parent::table", 10)
                dtable_html = dtable_element.get_attribute('outerHTML')
                odrives = pd.read_html(otable_html)[0]
                ddrives = pd.read_html(dtable_html)[0]
                odrives['opponent'] = opponent_nickname
                ddrives['opponent'] = opponent_nickname
                odrives['date'] = d
                ddrives['date'] = d
                odrives['points'] = p #
                ddrives['points_allowed'] = pa #
                odrives['date'] = odrives['date'].astype(str).str.rstrip('0')
                ddrives['date'] = ddrives['date'].astype(str).str.rstrip('0')
                odrives['date'] = pd.to_datetime(odrives['date'], format='%Y%m%d')
                ddrives['date'] = pd.to_datetime(ddrives['date'], format='%Y%m%d')
                odrives['outcome'] = r
                ddrives['outcome'] = r
                odrives['passing_yards'] = py #
                odrives['rushing_yards'] = ry #
                odrives['turnovers'] = t #
                offensive_drives = pd.concat([offensive_drives, odrives])
                defensive_drives = pd.concat([defensive_drives, ddrives])
            elif l == 'N':
                opponent_full_name = inverse_mapping[o]
                opponent_nickname = opponent_full_name.split()[-1]
                print(opponent_nickname)
                url = f'https://www.pro-football-reference.com/boxscores/{d}{o}.htm#home_drives'
                print(url)
                browser.get(url)
                otable_element = find_element_with_timeout(browser, By.XPATH, f"//caption[text()='{nickname} Drives Table']/parent::table", 10)
                otable_html = otable_element.get_attribute('outerHTML')
                dtable_element = find_element_with_timeout(browser, By.XPATH, f"//caption[text()='{opponent_nickname} Drives Table']/parent::table", 10)
                dtable_html = dtable_element.get_attribute('outerHTML')
                odrives = pd.read_html(otable_html)[0]
                ddrives = pd.read_html(dtable_html)[0]
                odrives['opponent'] = opponent_nickname
                ddrives['opponent'] = opponent_nickname
                odrives['date'] = d
                ddrives['date'] = d
                odrives['points'] = p #
                ddrives['points_allowed'] = pa #
                odrives['date'] = odrives['date'].astype(str).str.rstrip('0')
                ddrives['date'] = ddrives['date'].astype(str).str.rstrip('0')
                odrives['date'] = pd.to_datetime(odrives['date'], format='%Y%m%d')
                ddrives['date'] = pd.to_datetime(ddrives['date'], format='%Y%m%d')
                odrives['outcome'] = r
                ddrives['outcome'] = r
                odrives['passing_yards'] = py #
                odrives['rushing_yards'] = ry #
                odrives['turnovers'] = t #
                offensive_drives = pd.concat([offensive_drives, odrives])
                defensive_drives = pd.concat([defensive_drives, ddrives])
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Couldn't find the data for {nickname} on {d}. Possibly a future or postponed game.")
            continue
    
    browser.quit()
    
    
    # PER GAME LOOP
    
    offensive_game_stats = pd.DataFrame()
    defensive_game_stats = pd.DataFrame()
    total_offensive_drives = pd.DataFrame()
    total_defensive_drives = pd.DataFrame()
    
    for x in offensive_drives['date'].unique():
        single_game_offense = offensive_drives[(offensive_drives['date'] == x)]
        single_game_defense = defensive_drives[(defensive_drives['date'] == x)]
        
        def adjust_los(row, team_name):
            team_abbreviation, yardage = row['LOS'].split()
            yardage = int(yardage)
    
            if team_abbreviation == team_name.upper():
                return -yardage
            return yardage
    
        #offense
        
        single_game_offense['LOS'] = single_game_offense['LOS'].fillna(f'{team.upper()} 25')
        single_game_offense['Adj_LOS'] = single_game_offense.apply(lambda row: adjust_los(row, team), axis=1)
        single_game_offense = single_game_offense[~single_game_offense['Result'].isin(['End of Game', 'End of Half'])]
        single_game_offense['Length'] = pd.to_timedelta('00:' + single_game_offense['Length'])
        single_game_offense['yards_per_play'] = single_game_offense['Net Yds'] / single_game_offense['Plays'] 
        total_offensive_drives = pd.concat([total_offensive_drives, single_game_offense])
        total_offensive_drives.to_csv(f'{year} - {nickname} Offensive Drive Summary.csv')
    
        #defense
        
        single_game_defense['LOS'] = single_game_defense['LOS'].fillna(f'{team.upper()} 25')
        single_game_defense['Adj_LOS'] = single_game_defense.apply(lambda row: adjust_los(row, team), axis=1)
        single_game_defense = single_game_defense[~single_game_defense['Result'].isin(['End of Game', 'End of Half'])]
        single_game_defense['Length'] = pd.to_timedelta('00:' + single_game_defense['Length'])
        single_game_defense['yards_per_play'] = single_game_defense['Net Yds'] / single_game_defense['Plays'] 
        total_defensive_drives = pd.concat([total_defensive_drives, single_game_defense])
        total_defensive_drives.to_csv(f'{year} - {nickname} Defensive Drive Summary.csv')
    
        #offense
    
        def calculate_offensive_conversion_rates(single_game_offense):
            ototal_drive_count = len(single_game_offense)
            osuccessful_drives = len(single_game_offense[single_game_offense['Result'].isin(['Touchdown', 'Field Goal'])])
            otouchdown_drives = len(single_game_offense[single_game_offense['Result'] == 'Touchdown'])
            ofield_goal_drives = len(single_game_offense[single_game_offense['Result'] == 'Field Goal'])
            ocombined_conversion_rate = osuccessful_drives / ototal_drive_count
            otouchdown_conversion_rate = otouchdown_drives / ototal_drive_count
            ofield_goal_conversion_rate = ofield_goal_drives / ototal_drive_count
            return ocombined_conversion_rate, otouchdown_conversion_rate, ofield_goal_conversion_rate

        ocombined_rate, otd_rate, ofg_rate = calculate_offensive_conversion_rates(single_game_offense)
    
        #defense
    
        def calculate_defensive_conversion_rates(single_game_defense):
            dtotal_drive_count = len(single_game_defense)
            dsuccessful_drives = len(single_game_defense[single_game_defense['Result'].isin(['Touchdown', 'Field Goal'])])
            dtouchdown_drives = len(single_game_defense[single_game_defense['Result'] == 'Touchdown'])
            dfield_goal_drives = len(single_game_defense[single_game_defense['Result'] == 'Field Goal'])
            dcombined_conversion_rate = dsuccessful_drives / dtotal_drive_count
            dtouchdown_conversion_rate = dtouchdown_drives / dtotal_drive_count
            dfield_goal_conversion_rate = dfield_goal_drives / dtotal_drive_count
            return dcombined_conversion_rate, dtouchdown_conversion_rate, dfield_goal_conversion_rate
    
        dcombined_rate, dtd_rate, dfg_rate = calculate_offensive_conversion_rates(single_game_defense)

        
        offensive_stat_dict = {'team':[], 'opponent':[], 'date':[], 'outcome':[], 'points':[],  'total_plays':[], 'total_yards':[], 'passing_yards':[], 'rushing_yards':[], 'turnovers':[], 'avg_plays_drive':[], 'avg_yards_play':[], 'avg_yards_drive':[], 'avg_drive_length':[], 'ccr':[], 'tdcr':[], 'fgcr':[]}
        
        defensive_stat_dict = {'team':[], 'opponent':[], 'date':[], 'outcome':[], 'points_allowed':[], 'total_plays':[], 'total_yards':[], 'avg_plays_drive':[], 'avg_yards_play':[], 'avg_yards_drive':[], 'avg_drive_length':[], 'ccr':[], 'tdcr':[], 'fgcr':[]}
    
        team = nickname
        opponent = single_game_offense['opponent'].unique()[0]
        outcome = single_game_offense['outcome'].unique()[0]
        date = x
        points = single_game_offense['points'].unique()[0] #
        passing_yards = single_game_offense['passing_yards'].unique()[0] #
        rushing_yards = single_game_offense['rushing_yards'].unique()[0] #
        turnovers = single_game_offense['turnovers'].unique()[0] #
        ototal_plays = single_game_offense['Plays'].sum()
        ototal_yards = single_game_offense['Net Yds'].sum()
        oplays_drive = single_game_offense['Plays'].mean()
        oyards_play = single_game_offense['yards_per_play'].mean()
        oyards_drive = single_game_offense['Net Yds'].mean()
        odrive_length = single_game_offense['Length'].mean()
        occr = ocombined_rate
        otdcr = otd_rate
        ofgcr = ofg_rate
        offensive_stat_dict['team'].append(team)
        offensive_stat_dict['opponent'].append(opponent)
        offensive_stat_dict['date'].append(date)
        offensive_stat_dict['outcome'].append(outcome)
        offensive_stat_dict['points'].append(points)
        offensive_stat_dict['total_plays'].append(ototal_plays)
        offensive_stat_dict['total_yards'].append(ototal_yards)
        offensive_stat_dict['passing_yards'].append(passing_yards)
        offensive_stat_dict['rushing_yards'].append(rushing_yards)
        offensive_stat_dict['turnovers'].append(turnovers)
        offensive_stat_dict['avg_yards_play'].append(oyards_play)
        offensive_stat_dict['avg_plays_drive'].append(oplays_drive)
        offensive_stat_dict['avg_yards_drive'].append(oyards_drive)
        offensive_stat_dict['avg_drive_length'].append(odrive_length)
        offensive_stat_dict['ccr'].append(occr)
        offensive_stat_dict['tdcr'].append(otdcr)
        offensive_stat_dict['fgcr'].append(ofgcr)
        offensive_stat_dict = pd.DataFrame(offensive_stat_dict)
        offensive_game_stats = pd.concat([offensive_game_stats, offensive_stat_dict])
        #offensive_stat_dict.to_csv(f'{team} offense.csv')
        
        points_allowed = single_game_defense['points_allowed'].unique()[0] 
        dtotal_plays = single_game_defense['Plays'].sum()
        dtotal_yards = single_game_defense['Net Yds'].sum()
        dplays_drive = single_game_defense['Plays'].mean()
        dyards_play = single_game_defense['yards_per_play'].mean()
        dyards_drive = single_game_defense['Net Yds'].mean()
        ddrive_length = single_game_defense['Length'].mean()
        dccr = dcombined_rate
        dtdcr = dtd_rate
        dfgcr = dfg_rate
        defensive_stat_dict['team'].append(team)
        defensive_stat_dict['opponent'].append(opponent)
        defensive_stat_dict['date'].append(date)
        defensive_stat_dict['points_allowed'].append(points_allowed)
        defensive_stat_dict['outcome'].append(outcome)
        defensive_stat_dict['total_plays'].append(dtotal_plays)
        defensive_stat_dict['total_yards'].append(dtotal_yards)
        defensive_stat_dict['avg_yards_play'].append(dyards_play)
        defensive_stat_dict['avg_plays_drive'].append(dplays_drive)
        defensive_stat_dict['avg_yards_drive'].append(dyards_drive)
        defensive_stat_dict['avg_drive_length'].append(ddrive_length)
        defensive_stat_dict['ccr'].append(dccr)
        defensive_stat_dict['tdcr'].append(dtdcr)
        defensive_stat_dict['fgcr'].append(dfgcr)
        defensive_stat_dict = pd.DataFrame(defensive_stat_dict)
        defensive_game_stats = pd.concat([defensive_game_stats, defensive_stat_dict])
        #defensive_stat_dict.to_csv(f'{team} defense.csv')
        
    offensive_game_stats.to_csv(f'{nickname} offense.csv')
    defensive_game_stats.to_csv(f'{nickname} defense.csv')
    
    return offensive_drives, defensive_drives, offensive_game_stats, defensive_game_stats


#pass yards
#rush yards
#points
#turnovers


/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:11: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [49]:
odrives, ddrives, ostats, dstats = conversion_rate('pit', 2023)
display(ostats, dstats)

Scraping data for: Steelers
https://www.pro-football-reference.com/teams/pit/2023.htm
49ers
https://www.pro-football-reference.com/boxscores/202309100pit.htm#home_drives
Browns
https://www.pro-football-reference.com/boxscores/202309180pit.htm#home_drives
Raiders
https://www.pro-football-reference.com/boxscores/202309240rai.htm#home_drives
Texans
https://www.pro-football-reference.com/boxscores/202310010htx.htm#home_drives
Ravens
https://www.pro-football-reference.com/boxscores/202310080pit.htm#home_drives
Rams
https://www.pro-football-reference.com/boxscores/202310220ram.htm#home_drives
Jaguars
https://www.pro-football-reference.com/boxscores/202310290pit.htm#home_drives
Titans
https://www.pro-football-reference.com/boxscores/202311020pit.htm#home_drives
Packers
https://www.pro-football-reference.com/boxscores/202311120pit.htm#home_drives
Browns
https://www.pro-football-reference.com/boxscores/202311190cle.htm#home_drives
Couldn't find the data for Steelers on 202311190. Possibly a fut

/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['LOS'] = single_game_offense['LOS'].fillna(f'{team.upper()} 25')
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['Adj_LOS'] = single_game_offense.apply(lambda row: adjust_los(row, team), axis=1)
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/98731

,team,opponent,date,outcome,points,total_plays,total_yards,passing_yards,rushing_yards,turnovers,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Steelers,49ers,2023-09-01,L,7.0,61,244,198.0,41.0,2.0,5.545455,2.612013,22.181818,0 days 00:02:03.363636363,0.090909,0.090909,0.000000
0,Steelers,Browns,2023-09-18,W,26.0,52,248,200.0,55.0,2.0,4.333333,3.806019,20.666667,0 days 00:01:55.833333333,0.250000,0.083333,0.166667
0,Steelers,Raiders,2023-09-24,W,23.0,61,354,228.0,105.0,NaN,5.545455,6.047291,32.181818,0 days 00:02:40,0.454545,0.181818,0.272727
0,Steelers,Texans,2023-10-01,L,6.0,51,204,111.0,114.0,1.0,5.666667,3.482143,22.666667,0 days 00:02:38.444444444,0.222222,0.000000,0.222222
0,Steelers,Ravens,2023-10-08,W,17.0,66,291,202.0,87.0,1.0,6.600000,3.770873,29.100000,0 days 00:02:55.800000,0.400000,0.100000,0.300000
0,Steelers,Rams,2023-10-22,W,24.0,46,212,214.0,86.0,NaN,5.111111,3.683333,23.555556,0 days 00:02:29.888888888,0.444444,0.333333,0.111111
0,Steelers,Jaguars,2023-10-29,L,10.0,66,288,191.0,70.0,2.0,5.500000,2.999306,24.000000,0 days 00:02:08.666666666,0.166667,0.083333,0.083333
0,Steelers,Titans,2023-11-02,W,20.0,61,335,160.0,166.0,NaN,7.625000,4.103788,41.875000,0 days 00:03:35.625000,0.500000,0.250000,0.250000
0,Steelers,Packers,2023-11-12,W,23.0,63,331,119.0,205.0,NaN,6.300000,4.662955,33.100000,0 days 00:03:03.300000,0.500000,0.200000,0.300000


,team,opponent,date,outcome,points_allowed,total_plays,total_yards,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Steelers,49ers,2023-09-01,L,30.0,66,384,6.600000,7.053535,38.400000,0 days 00:03:36.300000,0.600000,0.300000,0.300000
0,Steelers,Browns,2023-09-18,W,22.0,84,358,5.600000,3.682100,23.866667,0 days 00:02:21.866666666,0.266667,0.133333,0.133333
0,Steelers,Raiders,2023-09-24,W,18.0,68,362,5.666667,4.206226,30.166667,0 days 00:02:29.750000,0.250000,0.166667,0.083333
0,Steelers,Texans,2023-10-01,L,30.0,72,416,7.200000,5.304199,41.600000,0 days 00:03:15.500000,0.600000,0.300000,0.300000
0,Steelers,Ravens,2023-10-08,W,10.0,68,320,5.666667,4.231043,26.666667,0 days 00:02:31.166666666,0.166667,0.083333,0.083333
0,Steelers,Rams,2023-10-22,W,17.0,65,339,6.500000,4.283788,33.900000,0 days 00:03:10.600000,0.300000,0.200000,0.100000
0,Steelers,Jaguars,2023-10-29,L,20.0,69,384,6.272727,6.712698,34.909091,0 days 00:03:06.909090909,0.454545,0.090909,0.363636
0,Steelers,Titans,2023-11-02,W,16.0,71,372,7.888889,4.618987,41.333333,0 days 00:03:27.666666666,0.444444,0.111111,0.333333
0,Steelers,Packers,2023-11-12,W,19.0,65,396,6.500000,5.198802,39.600000,0 days 00:02:48.700000,0.400000,0.200000,0.200000


In [51]:
odrives, ddrives, ostats, dstats = conversion_rate('mia', 2023)
display(ostats, dstats)

Scraping data for: Dolphins
https://www.pro-football-reference.com/teams/mia/2023.htm
Chargers
https://www.pro-football-reference.com/boxscores/202309100sdg.htm#home_drives
Patriots
https://www.pro-football-reference.com/boxscores/202309170nwe.htm#home_drives
Broncos
https://www.pro-football-reference.com/boxscores/202309240mia.htm#home_drives
Bills
https://www.pro-football-reference.com/boxscores/202310010buf.htm#home_drives
Giants
https://www.pro-football-reference.com/boxscores/202310080mia.htm#home_drives
Panthers
https://www.pro-football-reference.com/boxscores/202310150mia.htm#home_drives
Eagles
https://www.pro-football-reference.com/boxscores/202310220phi.htm#home_drives
Patriots
https://www.pro-football-reference.com/boxscores/202310290mia.htm#home_drives
Chiefs
https://www.pro-football-reference.com/boxscores/202311050kan.htm#home_drives
Raiders
https://www.pro-football-reference.com/boxscores/202311190mia.htm#home_drives
Couldn't find the data for Dolphins on 202311190. Possi

/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['LOS'] = single_game_offense['LOS'].fillna(f'{team.upper()} 25')
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['Adj_LOS'] = single_game_offense.apply(lambda row: adjust_los(row, team), axis=1)
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/98731

,team,opponent,date,outcome,points,total_plays,total_yards,passing_yards,rushing_yards,turnovers,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Dolphins,Chargers,2023-09-01,W,36.0,66,563,466.0,70.0,2.0,6.600000,12.161944,56.300000,0 days 00:02:38.400000,0.700000,0.400000,0.300
0,Dolphins,Patriots,2023-09-17,W,24.0,62,388,244.0,145.0,1.0,6.200000,8.278759,38.800000,0 days 00:02:56.800000,0.400000,0.300000,0.100
0,Dolphins,Broncos,2023-09-24,W,70.0,70,706,376.0,350.0,NaN,5.384615,11.728632,54.307692,0 days 00:02:33.692307692,0.769231,0.769231,0.000
0,Dolphins,Bills,2023-10-01,L,20.0,58,384,251.0,142.0,2.0,5.272727,5.169618,34.909091,0 days 00:02:41.636363636,0.272727,0.272727,0.000
0,Dolphins,Giants,2023-10-08,W,31.0,51,500,302.0,222.0,3.0,5.100000,10.823889,50.000000,0 days 00:02:16,0.500000,0.400000,0.100
0,Dolphins,Panthers,2023-10-15,W,42.0,62,424,262.0,162.0,1.0,5.636364,5.896212,38.545455,0 days 00:02:29.363636363,0.545455,0.545455,0.000
0,Dolphins,Eagles,2023-10-22,L,17.0,49,204,199.0,45.0,1.0,6.125000,3.717857,25.500000,0 days 00:02:54.625000,0.250000,0.125000,0.125
0,Dolphins,Patriots,2023-10-29,W,31.0,71,379,312.0,78.0,2.0,7.100000,4.302381,37.900000,0 days 00:03:24.300000,0.500000,0.400000,0.100
0,Dolphins,Chiefs,2023-11-05,L,14.0,58,272,175.0,117.0,1.0,5.272727,4.100000,24.727273,0 days 00:02:45.818181818,0.181818,0.181818,0.000


,team,opponent,date,outcome,points_allowed,total_plays,total_yards,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Dolphins,Chargers,2023-09-01,W,34.0,78,435,7.800000,4.486324,43.500000,0 days 00:03:16.100000,0.600000,0.400000,0.200000
0,Dolphins,Patriots,2023-09-17,W,17.0,71,305,7.100000,3.693333,30.500000,0 days 00:02:57,0.300000,0.200000,0.100000
0,Dolphins,Broncos,2023-09-24,W,20.0,60,341,5.000000,3.947090,28.416667,0 days 00:02:09.500000,0.250000,0.083333,0.166667
0,Dolphins,Bills,2023-10-01,L,48.0,49,446,4.900000,10.574643,44.600000,0 days 00:02:28.500000,0.800000,0.600000,0.200000
0,Dolphins,Giants,2023-10-08,W,16.0,72,241,8.000000,3.102920,26.777778,0 days 00:03:59.888888888,0.333333,0.000000,0.333333
0,Dolphins,Panthers,2023-10-15,W,21.0,69,306,6.272727,3.588745,27.818182,0 days 00:02:47.636363636,0.181818,0.181818,0.000000
0,Dolphins,Eagles,2023-10-22,L,31.0,63,380,7.875000,4.506757,47.500000,0 days 00:04:05.625000,0.625000,0.500000,0.125000
0,Dolphins,Patriots,2023-10-29,W,17.0,51,190,5.100000,2.759188,19.000000,0 days 00:02:21.900000,0.300000,0.200000,0.100000
0,Dolphins,Chiefs,2023-11-05,L,21.0,53,249,5.888889,3.709320,27.666667,0 days 00:03:09.777777777,0.222222,0.222222,0.000000


In [52]:
odrives, ddrives, ostats, dstats = conversion_rate('rav', 2023)
display(ostats, dstats)

Scraping data for: Ravens
https://www.pro-football-reference.com/teams/rav/2023.htm
Texans
https://www.pro-football-reference.com/boxscores/202309100rav.htm#home_drives
Bengals
https://www.pro-football-reference.com/boxscores/202309170cin.htm#home_drives
Colts
https://www.pro-football-reference.com/boxscores/202309240rav.htm#home_drives
Browns
https://www.pro-football-reference.com/boxscores/202310010cle.htm#home_drives
Steelers
https://www.pro-football-reference.com/boxscores/202310080pit.htm#home_drives
Titans
https://www.pro-football-reference.com/boxscores/202310150oti.htm#home_drives
Lions
https://www.pro-football-reference.com/boxscores/202310220rav.htm#home_drives
Cardinals
https://www.pro-football-reference.com/boxscores/202310290crd.htm#home_drives
Seahawks
https://www.pro-football-reference.com/boxscores/202311050rav.htm#home_drives
Browns
https://www.pro-football-reference.com/boxscores/202311120rav.htm#home_drives
Bengals
https://www.pro-football-reference.com/boxscores/202

/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['LOS'] = single_game_offense['LOS'].fillna(f'{team.upper()} 25')
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['Adj_LOS'] = single_game_offense.apply(lambda row: adjust_los(row, team), axis=1)
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/98731

,team,opponent,date,outcome,points,total_plays,total_yards,passing_yards,rushing_yards,turnovers,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Ravens,Texans,2023-09-01,W,25.0,59,263,155.0,110.0,2.0,5.363636,3.850253,23.909091,0 days 00:02:48.636363636,0.363636,0.272727,0.090909
0,Ravens,Bengals,2023-09-17,W,27.0,66,363,237.0,178.0,NaN,8.250000,5.743773,45.375000,0 days 00:03:42,0.625000,0.375000,0.250000
0,Ravens,Colts,2023-09-24,L,19.0,73,339,178.0,186.0,2.0,5.214286,3.397024,24.214286,0 days 00:02:17.142857142,0.214286,0.142857,0.071429
0,Ravens,Browns,2023-10-01,W,28.0,55,265,165.0,131.0,1.0,4.583333,3.933829,22.083333,0 days 00:02:29.166666666,0.333333,0.333333,0.000000
0,Ravens,Steelers,2023-10-08,L,10.0,68,320,210.0,125.0,3.0,5.666667,4.231043,26.666667,0 days 00:02:31.166666666,0.166667,0.083333,0.083333
0,Ravens,Titans,2023-10-15,W,24.0,75,379,221.0,139.0,1.0,6.818182,3.537884,34.454545,0 days 00:03:11.272727272,0.636364,0.090909,0.545455
0,Ravens,Lions,2023-10-22,W,38.0,55,480,357.0,146.0,1.0,6.111111,8.865440,53.333333,0 days 00:03:21.888888888,0.666667,0.555556,0.111111
0,Ravens,Cardinals,2023-10-29,W,31.0,61,304,138.0,130.0,NaN,5.545455,4.178680,27.636364,0 days 00:02:39.363636363,0.454545,0.363636,0.090909
0,Ravens,Seahawks,2023-11-05,W,37.0,78,518,217.0,298.0,2.0,6.500000,7.749106,43.166667,0 days 00:03:20.333333333,0.583333,0.333333,0.250000
0,Ravens,Browns,2023-11-12,L,31.0,52,311,200.0,106.0,2.0,5.200000,5.545873,31.100000,0 days 00:02:34.500000,0.400000,0.300000,0.100000


,team,opponent,date,outcome,points_allowed,total_plays,total_yards,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Ravens,Texans,2023-09-01,W,9.0,69,244,6.272727,2.722481,22.181818,0 days 00:02:33.363636363,0.272727,0.000000,0.272727
0,Ravens,Bengals,2023-09-17,W,24.0,58,288,8.285714,3.788462,41.142857,0 days 00:03:50.857142857,0.428571,0.285714,0.142857
0,Ravens,Colts,2023-09-24,L,22.0,88,312,5.866667,3.164074,20.800000,0 days 00:02:26.133333333,0.400000,0.066667,0.333333
0,Ravens,Browns,2023-10-01,W,3.0,65,189,5.416667,2.503175,15.750000,0 days 00:02:29.916666666,0.083333,0.000000,0.083333
0,Ravens,Steelers,2023-10-08,L,17.0,66,291,6.600000,3.770873,29.100000,0 days 00:02:55.800000,0.400000,0.100000,0.300000
0,Ravens,Titans,2023-10-15,W,16.0,49,292,4.900000,6.558492,29.200000,0 days 00:02:25.800000,0.400000,0.100000,0.300000
0,Ravens,Lions,2023-10-22,W,6.0,64,294,7.111111,3.899639,32.666667,0 days 00:02:56.666666666,0.111111,0.111111,0.000000
0,Ravens,Cardinals,2023-10-29,W,24.0,71,328,5.916667,3.381944,27.333333,0 days 00:02:30.083333333,0.333333,0.250000,0.083333
0,Ravens,Seahawks,2023-11-05,W,3.0,47,137,4.272727,1.683766,12.454545,0 days 00:01:46,0.090909,0.000000,0.090909
0,Ravens,Browns,2023-11-12,L,33.0,79,387,7.900000,4.454986,38.700000,0 days 00:03:25.500000,0.600000,0.200000,0.400000


In [53]:
odrives, ddrives, ostats, dstats = conversion_rate('cle', 2023)
display(ostats, dstats)

Scraping data for: Browns
https://www.pro-football-reference.com/teams/cle/2023.htm
Bengals
https://www.pro-football-reference.com/boxscores/202309100cle.htm#home_drives
Steelers
https://www.pro-football-reference.com/boxscores/202309180pit.htm#home_drives
Titans
https://www.pro-football-reference.com/boxscores/202309240cle.htm#home_drives
Ravens
https://www.pro-football-reference.com/boxscores/202310010cle.htm#home_drives
49ers
https://www.pro-football-reference.com/boxscores/202310150cle.htm#home_drives
Colts
https://www.pro-football-reference.com/boxscores/202310220clt.htm#home_drives
Seahawks
https://www.pro-football-reference.com/boxscores/202310290sea.htm#home_drives
Cardinals
https://www.pro-football-reference.com/boxscores/202311050cle.htm#home_drives
Ravens
https://www.pro-football-reference.com/boxscores/202311120rav.htm#home_drives
Steelers
https://www.pro-football-reference.com/boxscores/202311190cle.htm#home_drives
Couldn't find the data for Browns on 202311190. Possibly a

/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['LOS'] = single_game_offense['LOS'].fillna(f'{team.upper()} 25')
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['Adj_LOS'] = single_game_offense.apply(lambda row: adjust_los(row, team), axis=1)
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/98731

,team,opponent,date,outcome,points,total_plays,total_yards,passing_yards,rushing_yards,turnovers,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Browns,Bengals,2023-09-01,W,24.0,75,330,144.0,206.0,2.0,5.357143,4.106859,23.571429,0 days 00:02:33.571428571,0.357143,0.142857,0.214286
0,Browns,Steelers,2023-09-18,L,22.0,84,358,210.0,198.0,4.0,5.600000,3.682100,23.866667,0 days 00:02:21.866666666,0.266667,0.133333,0.133333
0,Browns,Titans,2023-09-24,W,27.0,69,391,263.0,78.0,1.0,7.666667,5.210041,43.444444,0 days 00:04:16.555555555,0.555556,0.333333,0.222222
0,Browns,Ravens,2023-10-01,L,3.0,65,189,73.0,93.0,3.0,5.416667,2.503175,15.750000,0 days 00:02:29.916666666,0.083333,0.000000,0.083333
0,Browns,49ers,2023-10-15,W,19.0,74,320,174.0,160.0,2.0,6.166667,3.480952,26.666667,0 days 00:02:44.750000,0.416667,0.083333,0.333333
0,Browns,Colts,2023-10-22,W,39.0,75,330,166.0,150.0,2.0,5.000000,4.068571,22.000000,0 days 00:01:49.800000,0.466667,0.200000,0.266667
0,Browns,Seahawks,2023-10-29,L,20.0,76,388,230.0,155.0,3.0,6.909091,4.797875,35.272727,0 days 00:03:18,0.363636,0.181818,0.181818
0,Browns,Cardinals,2023-11-05,W,27.0,74,331,213.0,113.0,NaN,5.285714,4.061039,23.642857,0 days 00:02:36.071428571,0.357143,0.214286,0.142857
0,Browns,Ravens,2023-11-12,W,33.0,79,387,195.0,178.0,2.0,7.900000,4.454986,38.700000,0 days 00:03:25.500000,0.600000,0.200000,0.400000


,team,opponent,date,outcome,points_allowed,total_plays,total_yards,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Browns,Bengals,2023-09-01,W,3.0,54,147,4.153846,2.380403,11.307692,0 days 00:01:48.384615384,0.076923,0.000000,0.076923
0,Browns,Steelers,2023-09-18,L,26.0,52,248,4.333333,3.806019,20.666667,0 days 00:01:55.833333333,0.250000,0.083333,0.166667
0,Browns,Titans,2023-09-24,W,3.0,34,58,4.250000,1.223512,7.250000,0 days 00:02:25.625000,0.125000,0.000000,0.125000
0,Browns,Ravens,2023-10-01,L,28.0,55,265,4.583333,3.933829,22.083333,0 days 00:02:29.166666666,0.333333,0.333333,0.000000
0,Browns,49ers,2023-10-15,W,17.0,57,224,4.750000,2.748906,18.666667,0 days 00:02:13,0.250000,0.166667,0.083333
0,Browns,Colts,2023-10-22,W,38.0,69,451,4.312500,5.853423,28.187500,0 days 00:02:01.937500,0.375000,0.312500,0.062500
0,Browns,Seahawks,2023-10-29,L,24.0,55,357,5.000000,5.424892,32.454545,0 days 00:02:06,0.363636,0.272727,0.090909
0,Browns,Cardinals,2023-11-05,W,0.0,45,59,3.461538,0.635897,4.538462,0 days 00:01:40.461538461,0.000000,0.000000,0.000000
0,Browns,Ravens,2023-11-12,W,31.0,52,311,5.200000,5.545873,31.100000,0 days 00:02:34.500000,0.400000,0.300000,0.100000


In [55]:
odrives, ddrives, ostats, dstats = conversion_rate('kan', 2023)
display(ostats, dstats)

Scraping data for: Chiefs
https://www.pro-football-reference.com/teams/kan/2023.htm
Lions
https://www.pro-football-reference.com/boxscores/202309070kan.htm#home_drives
Jaguars
https://www.pro-football-reference.com/boxscores/202309170jax.htm#home_drives
Bears
https://www.pro-football-reference.com/boxscores/202309240kan.htm#home_drives
Jets
https://www.pro-football-reference.com/boxscores/202310010nyj.htm#home_drives
Vikings
https://www.pro-football-reference.com/boxscores/202310080min.htm#home_drives
Broncos
https://www.pro-football-reference.com/boxscores/202310120kan.htm#home_drives
Chargers
https://www.pro-football-reference.com/boxscores/202310220kan.htm#home_drives
Broncos
https://www.pro-football-reference.com/boxscores/202310290den.htm#home_drives
Dolphins
https://www.pro-football-reference.com/boxscores/202311050kan.htm#home_drives
Eagles
https://www.pro-football-reference.com/boxscores/202311200kan.htm#home_drives
Couldn't find the data for Chiefs on 202311200. Possibly a fut

/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['LOS'] = single_game_offense['LOS'].fillna(f'{team.upper()} 25')
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['Adj_LOS'] = single_game_offense.apply(lambda row: adjust_los(row, team), axis=1)
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/98731

,team,opponent,date,outcome,points,total_plays,total_yards,passing_yards,rushing_yards,turnovers,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Chiefs,Lions,2023-09-07,L,20.0,64,332,226.0,90.0,1.0,5.818182,4.168415,30.181818,0 days 00:02:29.272727272,0.363636,0.181818,0.181818
0,Chiefs,Jaguars,2023-09-17,W,17.0,55,274,298.0,101.0,3.0,6.111111,3.884018,30.444444,0 days 00:03:03.444444444,0.333333,0.222222,0.111111
0,Chiefs,Bears,2023-09-24,W,41.0,70,434,303.0,153.0,2.0,7.000000,5.877424,43.400000,0 days 00:03:11.300000,0.700000,0.500000,0.200000
0,Chiefs,Jets,2023-10-01,W,23.0,54,361,197.0,204.0,2.0,5.400000,4.807262,36.100000,0 days 00:02:28.500000,0.500000,0.200000,0.300000
0,Chiefs,Vikings,2023-10-08,W,27.0,66,339,266.0,67.0,NaN,8.250000,4.545265,42.375000,0 days 00:03:59.875000,0.625000,0.375000,0.250000
0,Chiefs,Broncos,2023-10-12,W,19.0,71,379,293.0,96.0,1.0,7.100000,5.522330,37.900000,0 days 00:03:24.700000,0.500000,0.100000,0.400000
0,Chiefs,Chargers,2023-10-22,W,31.0,63,469,415.0,68.0,2.0,6.300000,7.974603,46.900000,0 days 00:03:03.300000,0.500000,0.400000,0.100000
0,Chiefs,Broncos,2023-10-29,L,9.0,52,244,212.0,62.0,5.0,5.777778,4.208838,27.111111,0 days 00:02:25.888888888,0.333333,0.000000,0.333333
0,Chiefs,Dolphins,2023-11-05,W,21.0,53,249,174.0,93.0,1.0,5.888889,3.709320,27.666667,0 days 00:03:09.777777777,0.222222,0.222222,0.000000


,team,opponent,date,outcome,points_allowed,total_plays,total_yards,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Chiefs,Lions,2023-09-07,L,21.0,64,329,6.400000,4.117403,32.900000,0 days 00:03:03.800000,0.200000,0.200000,0.000000
0,Chiefs,Jaguars,2023-09-17,W,9.0,67,287,6.090909,2.970904,26.090909,0 days 00:02:33.727272727,0.272727,0.000000,0.272727
0,Chiefs,Bears,2023-09-24,W,10.0,52,193,5.200000,2.797035,19.300000,0 days 00:02:22.700000,0.200000,0.100000,0.100000
0,Chiefs,Jets,2023-10-01,W,20.0,56,330,5.600000,5.515079,33.000000,0 days 00:02:45.300000,0.300000,0.200000,0.100000
0,Chiefs,Vikings,2023-10-08,W,20.0,63,322,9.000000,6.074376,46.000000,0 days 00:03:50.571428571,0.571429,0.285714,0.285714
0,Chiefs,Broncos,2023-10-12,W,8.0,49,225,4.900000,4.377778,22.500000,0 days 00:02:25.100000,0.100000,0.100000,0.000000
0,Chiefs,Chargers,2023-10-22,W,17.0,61,343,6.100000,5.322525,34.300000,0 days 00:02:48.500000,0.300000,0.200000,0.100000
0,Chiefs,Broncos,2023-10-29,L,24.0,67,253,6.090909,3.899334,23.000000,0 days 00:03:04.272727272,0.363636,0.272727,0.090909
0,Chiefs,Dolphins,2023-11-05,W,14.0,58,272,5.272727,4.100000,24.727273,0 days 00:02:45.818181818,0.181818,0.181818,0.000000


In [56]:
odrives, ddrives, ostats, dstats = conversion_rate('jax', 2023)
display(ostats, dstats)

Scraping data for: Jaguars
https://www.pro-football-reference.com/teams/jax/2023.htm
Colts
https://www.pro-football-reference.com/boxscores/202309100clt.htm#home_drives
Chiefs
https://www.pro-football-reference.com/boxscores/202309170jax.htm#home_drives
Texans
https://www.pro-football-reference.com/boxscores/202309240jax.htm#home_drives
Falcons
https://www.pro-football-reference.com/boxscores/202310010jax.htm#home_drives
Bills
https://www.pro-football-reference.com/boxscores/202310080buf.htm#home_drives
Colts
https://www.pro-football-reference.com/boxscores/202310150jax.htm#home_drives
Saints
https://www.pro-football-reference.com/boxscores/202310190nor.htm#home_drives
Steelers
https://www.pro-football-reference.com/boxscores/202310290pit.htm#home_drives
49ers
https://www.pro-football-reference.com/boxscores/202311120jax.htm#home_drives
Titans
https://www.pro-football-reference.com/boxscores/202311190jax.htm#home_drives
Couldn't find the data for Jaguars on 202311190. Possibly a future

/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['LOS'] = single_game_offense['LOS'].fillna(f'{team.upper()} 25')
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/987317529.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_game_offense['Adj_LOS'] = single_game_offense.apply(lambda row: adjust_los(row, team), axis=1)
/var/folders/pq/lv0_zg713md9s1yr4csf20pw0000gp/T/ipykernel_3781/98731

,team,opponent,date,outcome,points,total_plays,total_yards,passing_yards,rushing_yards,turnovers,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Jaguars,Colts,2023-09-01,W,31.0,68,328,237.0,105.0,2.0,4.857143,3.031236,23.428571,0 days 00:02:20.857142857,0.357143,0.285714,0.071429
0,Jaguars,Chiefs,2023-09-17,L,9.0,67,287,197.0,74.0,1.0,6.090909,2.970904,26.090909,0 days 00:02:33.727272727,0.272727,0.000000,0.272727
0,Jaguars,Texans,2023-09-24,L,17.0,67,375,288.0,116.0,2.0,6.700000,5.620260,37.500000,0 days 00:02:59.300000,0.300000,0.200000,0.100000
0,Jaguars,Falcons,2023-10-01,W,23.0,59,309,195.0,105.0,NaN,7.375000,4.643750,38.625000,0 days 00:03:48.125000,0.500000,0.125000,0.375000
0,Jaguars,Bills,2023-10-08,W,25.0,82,494,278.0,196.0,2.0,7.454545,6.042743,44.909091,0 days 00:03:27.181818181,0.363636,0.272727,0.090909
0,Jaguars,Colts,2023-10-15,W,37.0,61,254,148.0,85.0,1.0,5.083333,3.909010,21.166667,0 days 00:02:14.166666666,0.583333,0.333333,0.250000
0,Jaguars,Saints,2023-10-19,W,31.0,54,348,217.0,113.0,2.0,4.909091,7.613420,31.636364,0 days 00:02:08.090909090,0.363636,0.272727,0.090909
0,Jaguars,Steelers,2023-10-29,W,20.0,69,384,271.0,106.0,3.0,6.272727,6.712698,34.909091,0 days 00:03:06.909090909,0.454545,0.090909,0.363636
0,Jaguars,49ers,2023-11-12,L,3.0,55,239,162.0,59.0,4.0,5.500000,2.879762,23.900000,0 days 00:02:40.900000,0.100000,0.000000,0.100000


,team,opponent,date,outcome,points_allowed,total_plays,total_yards,avg_plays_drive,avg_yards_play,avg_yards_drive,avg_drive_length,ccr,tdcr,fgcr
0,Jaguars,Colts,2023-09-01,W,21.0,66,279,5.076923,3.243933,21.461538,0 days 00:01:56.923076923,0.153846,0.153846,0.000000
0,Jaguars,Chiefs,2023-09-17,L,17.0,55,274,6.111111,3.884018,30.444444,0 days 00:03:03.444444444,0.333333,0.222222,0.111111
0,Jaguars,Texans,2023-09-24,L,37.0,59,341,5.900000,6.134603,34.100000,0 days 00:02:46.100000,0.600000,0.300000,0.300000
0,Jaguars,Falcons,2023-10-01,W,7.0,57,282,5.700000,4.002500,28.200000,0 days 00:02:42.900000,0.100000,0.100000,0.000000
0,Jaguars,Bills,2023-10-08,W,20.0,53,373,4.818182,7.097107,33.909091,0 days 00:01:58,0.272727,0.272727,0.000000
0,Jaguars,Colts,2023-10-15,W,20.0,77,356,5.923077,3.787164,27.384615,0 days 00:02:24.538461538,0.307692,0.153846,0.153846
0,Jaguars,Saints,2023-10-19,W,24.0,90,420,6.923077,4.671707,32.307692,0 days 00:02:44.461538461,0.384615,0.153846,0.230769
0,Jaguars,Steelers,2023-10-29,W,10.0,66,288,5.500000,2.999306,24.000000,0 days 00:02:08.666666666,0.166667,0.083333,0.083333
0,Jaguars,49ers,2023-11-12,L,34.0,59,422,5.900000,9.222329,42.200000,0 days 00:03:08.800000,0.600000,0.400000,0.200000


In [ ]:
# pass yards
# rush yards
# turnovers
# points